# Segmentation d'image avec 12 features

In [42]:
import os
import keras.models as models
#import keras.layers.containers as containers
from keras.layers.core import Layer, Dense, Dropout, Activation, Flatten, Reshape,  Permute
from keras.layers.convolutional import Convolution2D, MaxPooling2D, UpSampling2D, ZeroPadding2D
from keras.layers.normalization import BatchNormalization
from keras.models import Sequential
import pandas as pd
import tensorflow as tf
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import cv2
#os.environ['KERAS_BACKEND'] = 'theano'
# os.environ['KERAS_BACKEND']="tensorflow"
# os.environ['THEANO_FLAGS']='mode=FAST_RUN,device=gpu0,floatX=float32,optimizer=fast_compile'


In [16]:
os.chdir("CamVid")
os.chdir("train")
train_images=os.listdir()
os.chdir("..")
os.chdir("trainannot")
train_targets = os.listdir()
os.chdir("..")
os.chdir("..")

In [33]:
def binarylab(labels):
    x = np.zeros([360,480,12])    
    for i in range(360):
        for j in range(480):
            x[i,j,labels[i][j]]=1
    return x

In [63]:
X_train=[]
Y_train=[]
for path_image in train_images[:50]:
    img=plt.imread("./CamVid/train"+"/"+path_image)
    img = (img[:,:,0]+img[:,:,1]+img[:,:,2])/3
    img = np.expand_dims(img,axis=2)
    X_train.append(img)
    img=cv2.imread("./CamVid/trainannot"+"/"+path_image)
#     for i in range(img.shape[0]):
#         for j in range(img.shape[1]):
#             if 
    Y_train.append(np.expand_dims(np.concatenate(img[:,:,0]),axis=1))
    
#     img=cv2.resize(img,(224,224))
    

In [2]:
from __future__ import absolute_import
from __future__ import print_function
import matplotlib.pyplot as plt

#import pylab as pl
import matplotlib.cm as cm
import itertools
import numpy as np
#import theano as T

import tensorflow
import tensorflow as tf
#import theano.tensor as T
np.random.seed(1337) # for reproducibility
from keras.datasets import mnist
from keras.models import Sequential
#import keras.layers.containers as containers
from keras.layers.noise import GaussianNoise
import keras.models as models
#import keras.layers.containers as containers
from keras.layers.core import Layer, Dense, Dropout, Activation, Flatten, Reshape,  Permute
from keras.layers.convolutional import Convolution2D, MaxPooling2D, UpSampling2D, ZeroPadding2D
from keras.layers.normalization import BatchNormalization
from keras.utils import np_utils


from keras import backend as K

import cv2
import numpy as np

path = './CamVid/'
data_shape = 360*480

Using TensorFlow backend.


## Data Fetching 
I equalized histograms of rbg layers separately, so that change in lighting doesnt effect the results.

In [3]:
def normalized(rgb):
    #return rgb/255.0
    norm=np.zeros((rgb.shape[0], rgb.shape[1], 3),np.float32)

    b=rgb[:,:,0]
    g=rgb[:,:,1]
    r=rgb[:,:,2]

    norm[:,:,0]=cv2.equalizeHist(b)
    norm[:,:,1]=cv2.equalizeHist(g)
    norm[:,:,2]=cv2.equalizeHist(r)

    return norm

def binarylab(labels):
    x = np.zeros([360,480,12])    
    for i in range(360):
        for j in range(480):
            x[i,j,labels[i][j]]=1
    return x

def prep_data():
    train_data = []
    train_label = []
    import os
    with open(path+'train.txt') as f:
        txt = f.readlines()
        txt = [line.split(' ') for line in txt]
    print(len(txt))
    for i in range(367):#len(txt)
        ## these paths are very specific to my machine
        train_data.append(normalized(cv2.imread(os.getcwd() + txt[i][0][7:])))#np.rollaxis(,2)
        train_label.append(binarylab(cv2.imread(os.getcwd() + txt[i][1][7:][:-1])[:,:,0]))
        print('.',end='')
    return np.array(train_data), np.array(train_label)

train_data, train_label = prep_data()

train_label = np.reshape(train_label,(367,data_shape,12))#367


367
..................................................

### Creating and Compiling neuralnet

In [48]:
np.array(Y_train).shape

(50, 360, 480, 12)

In [ ]:
data_shape=360*480
def encoder():
        kernel = 3
        filter_size = 64
        pad = 1
        pool_size = 2
    
        E = Sequential()
        
        E.add(Layer(input_shape=(360, 480,1)))
        E.add(ZeroPadding2D(padding=(pad,pad)))
        
        E.add(Convolution2D(filter_size, (kernel, kernel), border_mode='valid'))
        E.add(BatchNormalization())
        E.add(Activation('relu'))
        E.add(MaxPooling2D(pool_size=(pool_size, pool_size)))
        
        E.add(ZeroPadding2D(padding=(pad,pad)))
        E.add(Convolution2D(128, kernel, kernel, border_mode='valid'))
        E.add(BatchNormalization())
        E.add(Activation('relu'))
        E.add(MaxPooling2D(pool_size=(pool_size, pool_size)))
        
        E.add(ZeroPadding2D(padding=(pad,pad)))
        E.add(Convolution2D(256, kernel, kernel, border_mode='valid'))
        E.add(BatchNormalization())
        E.add(Activation('relu'))
        E.add(MaxPooling2D(pool_size=(pool_size, pool_size)))

        E.add(ZeroPadding2D(padding=(pad,pad)))
        E.add(Convolution2D(512, kernel, kernel, border_mode='valid'))
        E.add(BatchNormalization())
        
        E.add(Activation('relu'))
        #E.summary()

        return E
    
def decoder():
    
        kernel = 3
        filter_size = 64
        pad = 1
        pool_size = 2
        
        D = Sequential()
        
        D.add(ZeroPadding2D(padding=(pad,pad)))
        D.add(Convolution2D(512, kernel, kernel, border_mode='valid'))
        D.add(BatchNormalization())

        #D.add(UnPooling2D(poolsize=(pool_size,pool_size)))
        D.add(UpSampling2D())
        
        D.add(ZeroPadding2D(padding=(pad,pad)))
        D.add(Convolution2D(256, kernel, kernel, border_mode='valid'))
        D.add(BatchNormalization())

        #D.add(UnPooling2D(poolsize=(pool_size,pool_size)))
        D.add(UpSampling2D())
        D.add(ZeroPadding2D(padding=(pad,pad)))
        D.add(Convolution2D(128, kernel, kernel, border_mode='valid'))
        D.add(BatchNormalization())

        #D.add(UnPooling2D(poolsize=(pool_size,pool_size)))
        D.add(UpSampling2D())
        D.add(ZeroPadding2D(padding=(pad,pad)))
        D.add(Convolution2D(filter_size, kernel, kernel, border_mode='valid'))
        D.add(BatchNormalization())
        #D.summary()
    
        return D
    
def reshapeData() :
    dataReshape = Sequential()
    dataReshape.add(Convolution2D(12, 1, 1, border_mode='valid',))
    dataReshape.add(Reshape((12,data_shape)))
    dataReshape.add(Permute((2, 1)))
    dataReshape.add(Activation('softmax'))
    return dataReshape
    
model = Sequential()
model.add(encoder())
model.add(decoder())
model.add(reshapeData())
model.summary()
#model.compile(loss="categorical_crossentropy", optimizer='adadelta')
model.compile(loss='sparse_categorical_crossentropy', # using the cross-entropy loss function
              optimizer='adam', # using the Adam optimiser
              metrics=['accuracy'])

nb_epoch = 5
batch_size = 16

history = model.fit(np.array(X_train), np.array(Y_train), batch_size=batch_size, nb_epoch=nb_epoch, verbose=1 )



C:\Users\Thomas\Anaconda3\envs\py36\lib\site-packages\ipykernel_launcher.py:13: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(64, (3, 3), padding="valid")`
  del sys.path[0]
C:\Users\Thomas\Anaconda3\envs\py36\lib\site-packages\ipykernel_launcher.py:19: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(128, (3, 3), padding="valid")`
C:\Users\Thomas\Anaconda3\envs\py36\lib\site-packages\ipykernel_launcher.py:25: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(256, (3, 3), padding="valid")`
C:\Users\Thomas\Anaconda3\envs\py36\lib\site-packages\ipykernel_launcher.py:31: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(512, (3, 3), padding="valid")`
C:\Users\Thomas\Anaconda3\envs\py36\lib\site-packages\ipykernel_launcher.py:49: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(512, (3, 3), padding="valid")`
C:\Users\Thomas\Anaconda3\envs\py36\lib\site-packages\ipykernel_launcher.py:56: UserWarning: 

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
sequential_26 (Sequential)   (None, 45, 60, 512)       1553664   
_________________________________________________________________
sequential_27 (Sequential)   (None, 360, 480, 64)      3912384   
_________________________________________________________________
sequential_28 (Sequential)   (None, 172800, 12)        780       
Total params: 5,466,828
Trainable params: 5,462,988
Non-trainable params: 3,840
_________________________________________________________________


C:\Users\Thomas\Anaconda3\envs\py36\lib\site-packages\ipykernel_launcher.py:95: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.


Epoch 1/5


In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline
plt.plot((history.history['loss']))
plt.plot((history.history['val_loss']))
plt.title('Model loss by epoch')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'valid'], loc='right')
plt.show()

plt.plot((history.history['loss']))
plt.plot((history.history['val_loss']))
plt.title('Model loss from 500 to 2000')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'valid'], loc='right')
plt.xlim((500, 2000))
plt.ylim((50000, 160000))
plt.show()